In [ ]:
import tensorflow as tf #tensorflow cuda ile cpu ve gpu kullanılabilecek sekilde yuklendi

In [ ]:
tf.test.is_gpu_available()
#gpu varmı tespit edildi

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices() # local cihazlar listelenir

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [ ]:
tf.debugging.set_log_device_placement(True)

# gpu calıstıgında calısacak tensörler yazılır
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]) #2 satırlı 2 sütünlü matrix
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]) 
c = tf.matmul(a, b)

print(c)

In [ ]:
import numpy as np

data=np.load('data.npy') # alınan resimler yüklenir
target=np.load('target.npy') #alınacak klasör labelları yüklenir


In [ ]:
from keras.models import Sequential,Model # binary sonuclar için kullanılan düz model
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D,Activation,MaxPooling2D
from keras.utils import normalize
from keras.layers import Concatenate
from keras import Input
from keras.callbacks import ModelCheckpoint
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


input_shape=data.shape[1:] #100,100,1 input shape olarak ayarlandı
inp=Input(shape=input_shape) #input model icin shape
convs=[]

parrallel_kernels=[3,5,7]

for k in range(len(parrallel_kernels)):

    conv = Conv2D(128, parrallel_kernels[k],padding='same',activation='relu',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)
 #evrişimli 2 boyutlu model ile 3,5,7 olarak label size belirlenir.Aynı olan matrisi kaydırma sayısı bulunur ve matris bulunana
out = Concatenate()(convs) # olusturulan layerlar eklenir 
conv_model = Model( inp, out) #evirisimli modelin inputu ve outputu belirlenir.

model = Sequential() #düz model alınır
model.add(conv_model) #evrişimsel sinir ağları seçilir

model.add(Conv2D(64,(3,3))) # input degeri 64 girilir ve matris layerları 3x3 olarak ayarlanır.
model.add(Activation('relu')) # pozitifse sonucu negatifse 0 veren sistem
model.add(MaxPooling2D(pool_size=(2,2))) #input 2x2 e indirildi

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) #girilen verinin boyutu azaltılıp kullanılabilecek hale getirilir
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu')) # yogunlastırma katmanları olur input boyutu girilir ve boyut azaltılır
model.add(Dropout(0.5)) #yeterli derecede basarılı olmayan kısımlar bırakılır
model.add(Dense(64,activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(2,input_dim=128,activation='softmax')) #softmax ile normalizasyon yapılır
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()
conv

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)
train_data.shape

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.1)
#en iyi verilen sonucları model checkpoint olarak kabul ederiz.
#daha önce kaydedilen checkpointler cagrılır

In [ ]:
print(model.evaluate(test_data,test_target))